In [1]:
from timelapse_tools import convert_to_mp4

In [2]:
img, dims = convert_to_mp4("timelapse_tools/tests/data/s_1_t_5_c_1_z_1.czi")
print(dims)
img

[2019-12-14 00:21:17,467] INFO - prefect.FlowRunner | Beginning Flow run for 'czi_to_mp4_conversion'
[2019-12-14 00:21:17,473] INFO - prefect.FlowRunner | Starting flow run.
[2019-12-14 00:21:17,497] INFO - prefect.TaskRunner | Task 'List': Starting task run...
[2019-12-14 00:21:17,506] INFO - prefect.TaskRunner | Task 'List': finished task run for task with final state: 'Success'
[2019-12-14 00:21:17,524] INFO - prefect.TaskRunner | Task 'List': Starting task run...
[2019-12-14 00:21:17,528] INFO - prefect.TaskRunner | Task 'List': finished task run for task with final state: 'Success'
[2019-12-14 00:21:17,559] INFO - prefect.TaskRunner | Task 'List': Starting task run...
[2019-12-14 00:21:17,567] INFO - prefect.TaskRunner | Task 'List': finished task run for task with final state: 'Success'
[2019-12-14 00:21:17,592] INFO - prefect.TaskRunner | Task 'List': Starting task run...
[2019-12-14 00:21:17,604] INFO - prefect.TaskRunner | Task 'List': finished task run for task with final sta

TypeError: cannot unpack non-iterable PosixPath object

In [ ]:
img, dims = convert_to_mp4("timelapse_tools/tests/data/s_1_t_5_c_1_z_1.czi", S=0)
print(dims)
img

In [ ]:
img, dims = convert_to_mp4("timelapse_tools/tests/data/s_1_t_5_c_1_z_1.czi", C=0)
print(dims)
img

In [ ]:
img, dims = convert_to_mp4("timelapse_tools/tests/data/s_None_t_5_c_1_z_None.czi")
print(dims)
img

In [ ]:
img, dims = convert_to_mp4("timelapse_tools/tests/data/s_None_t_5_c_1_z_None.czi", S=0)
print(dims)
img

In [ ]:
img, dims = convert_to_mp4("timelapse_tools/tests/data/s_None_t_5_c_1_z_None.czi", S=0, C=0)
print(dims)
img